# EOD: load from fundamentals API

- use the fundamentals API to get details about tickers and ETFS
- get ticker values from exchange lists and then load fundamentals from that ticker list



In [10]:
import pandas as pd
import numpy as np
import os
import requests
import json
from io import StringIO
import yaml
from datetime import date
import requests
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
plt.style.use("fivethirtyeight")
%matplotlib inline

# For reading stock data from yahoo
#import pandas_datareader as pdr
from pandas_datareader.data import DataReader
# import yahoo_fin.stock_info as si

# For time stamps
from datetime import datetime

# for LSTM
from keras.models import Sequential
from keras.layers import Dense, LSTM, Input
from keras.models import Model
# from tensorflow.keras.layers import Input, Dropout, Dense, BatchNormalization, Activation, concatenate, GRU, Embedding, Flatten, BatchNormalization
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model
# access datasets from quandl.com - need to pip install Quandl to use
import quandl
config_file = 'eod_test_config.yml'

In [11]:
# load config file
current_path = os.getcwd()
print("current directory is: "+current_path)

path_to_yaml = os.path.join(current_path, config_file)
print("path_to_yaml "+path_to_yaml)
try:
    with open (path_to_yaml, 'r') as c_file:
        config = yaml.safe_load(c_file)
except Exception as e:
    print('Error reading the config file')

current directory is: C:\personal\karma_stocks_2021\stock_investigation\notebooks
path_to_yaml C:\personal\karma_stocks_2021\stock_investigation\notebooks\eod_test_config.yml


In [12]:
# load config parms
parms = {}
eod_token = config['general']['eod_token']
eod_historical_token = config['general']['eod_historical_token']
fundamentals_directory = config['files']['fundamentals_directory']
exchange_list = config['exchange_json_list']

from_date = config['general']['master_start']
to_date = config['general']['master_end']
parms['master_date_mode'] = config['general']['master_date_mode']
parms['master_start'] = config['general']['master_start']
parms['master_end'] = config['general']['master_end']

In [13]:
# DataReader: https://riptutorial.com/pandas/topic/1912/pandas-datareader
# https://pandas-datareader.readthedocs.io/en/latest/remote_data.html
# 
# Set up End and Start times for data grab
# check to see if start and end dates are hard-coded with master dates
def set_start_end():
    if parms['master_date_mode']: # start and end hardcoded by parameters
        start = parms['master_start']
        end = parms['master_end']
    else: # end is current date; start is current date minus years_window
        end = datetime.now()
        start = datetime(end.year - parms['years_window'], end.month, end.day)
    # output a test dataset
    tester = DataReader('IBM', 'stooq', start, end)
    print(tester.shape)
    return(start,end)

In [14]:
def get_path():
    rawpath = os.getcwd()
    # data is in a directory that is a sibling to the directory containing the notebook
    path = os.path.abspath(os.path.join(rawpath, '..', fundamentals_directory))
    return(path)

# Batch load of EOD fundamentals data
- https://eodhistoricaldata.com/financial-apis/stock-etfs-fundamental-data-feeds/
- base_url = 'https://eodhistoricaldata.com/api/fundamentals/'+symbol

Steps:
- get fundamentals for an exchange, e.g. base_url = "https://eodhistoricaldata.com/api/fundamentals/GSPC.INDX"
- parse the resulting JSON to get list of tickers associated with that exchange
- get JSON dump from fundamentals API for each ticker in the list
- clean up JSON to (a) remove starting and ending quotes, (b) backslashes
- save JSON in timestamped JSON file


In [15]:
# get the JSON fundamentals dump for a given ticker

# https://eodhistoricaldata.com/api/fundamentals/AAPL.US?api_token=OeAFFmMliFG5orCUuwAKQ8l4WWFQ67YX
def get_eod_ticker_fundamentals(symbol='AAPL.US', api_token='OeAFFmMliFG5orCUuwAKQ8l4WWFQ67YX'):
    session = requests.Session()
    base_url = 'https://eodhistoricaldata.com/api/fundamentals/'+symbol
    params = {'api_token': api_token}
    r = session.get(base_url, params=params)
    if r.status_code == requests.codes.ok:
        df = pd.read_csv(StringIO(r.text), skipfooter=0,index_col=0)
        return(True, r)
    else:
        print("status code",str(r.status_code))
        print("reason code",str(r.reason))
        return(False,"null")
        #raise Exception(r.status_code, r.reason, url)

In [16]:
# write output JSON file with YMD timestamp

def write_json_file(ticker,output_path,json_struct):
    str_date = datetime.now().strftime("%Y-%m-%d")
    file_name = os.path.join(output_path,ticker+'_'+str_date+'.json')
    print("file name is: ",file_name)
    #json_object = json.dumps(json_struct, indent = 4)
    json_object = json_struct
    with open(file_name, 'w') as outfile:
        json.dump(json_object, outfile)
    #with open(file_name, "w") as outfile:
    #    outfile.write(json_object)

In [17]:
def get_ticker_dict_for_exchange(exchange_json):
    path = get_path()
    file_name = os.path.join(path,exchange_json)
    print("json file name is: ",file_name)
    f = open(file_name,)
    exchange_dict = json.load(f)
    ticker_dict = exchange_dict['Components']
    return(ticker_dict)

In [18]:
for exchange in exchange_list:
    ticker_dict = get_ticker_dict_for_exchange(exchange)
    #print("ticker_dict",ticker_dict)
    i = 0
    for ticker in ticker_dict:
        ticker_name = ticker_dict[ticker]['Code']
        ticker_exchange = ticker_dict[ticker]['Exchange']
        #print("ticker_name is: ",ticker_name)
        #print("ticker_exchange is: ",ticker_exchange)
        company = ticker_name+'.'+ticker_exchange
        print("company is: ",company)
        result_stat, r = get_eod_ticker_fundamentals(company,eod_token)
        # clean off the backslashes and starting and ending double quotes
        r_text = r.text
        json_output = json.loads(r.text)
        no_backslash = r_text.replace("\\", "")
        # my_str[:-1]
        no_end_quote = no_backslash[:-1]
        prepped_json = no_end_quote[1:]
        write_json_file(ticker_name,get_path(),json_output)
        i = i+1
        if i >= 3:
            break
    print("out of loop")
            
        

json file name is:  C:\personal\karma_stocks_2021\stock_investigation\static_load_fundamentals\gspc_clean.json
company is:  AIZ.US
file name is:  C:\personal\karma_stocks_2021\stock_investigation\static_load_fundamentals\AIZ_2021-10-09.json
company is:  MNST.US
file name is:  C:\personal\karma_stocks_2021\stock_investigation\static_load_fundamentals\MNST_2021-10-09.json
company is:  GPS.US
file name is:  C:\personal\karma_stocks_2021\stock_investigation\static_load_fundamentals\GPS_2021-10-09.json
out of loop


In [19]:
# single call
'''


company = 'SPY.US'
token = eod_token
start_date, end_date = set_start_end()
result_stat, r = get_eod_ticker_fundamentals(company,token)
path = get_path()
# text.split(sep, 1)[0]
ticker = company.split('.',1)[0]
print("ticker")
write_json_file(ticker,path,r.text)

'''

'\n\n\ncompany = \'SPY.US\'\ntoken = eod_token\nstart_date, end_date = set_start_end()\nresult_stat, r = get_eod_ticker_fundamentals(company,token)\npath = get_path()\n# text.split(sep, 1)[0]\nticker = company.split(\'.\',1)[0]\nprint("ticker")\nwrite_json_file(ticker,path,r.text)\n\n'

In [20]:
'''
# with open("sample.json", "w") as outfile:
#    outfile.write(json_object)

json_object = json.dumps(r.text, indent = 4)
with open("spy.json", "w") as outfile:
    outfile.write(json_object)
    '''

'\n# with open("sample.json", "w") as outfile:\n#    outfile.write(json_object)\n\njson_object = json.dumps(r.text, indent = 4)\nwith open("spy.json", "w") as outfile:\n    outfile.write(json_object)\n    '

In [21]:
# brute force call 
# https://eodhistoricaldata.com/api/fundamentals/AAPL.US?api_token=OeAFFmMliFG5orCUuwAKQ8l4WWFQ67YX
'''
session = requests.Session()
base_url = "https://eodhistoricaldata.com/api/fundamentals/GSPC.INDX"
api_token = "605766cf49ac93.89487471"
params = {'api_token': api_token}
r = session.get(base_url, params=params)
if r.status_code == requests.codes.ok:
    #df = pd.read_csv(StringIO(r.text), skipfooter=0, parse_dates=[0], index_col=0, engine='python')
    df = pd.read_csv(StringIO(r.text), skipfooter=0,index_col=0)

    
else:
    print("status code",str(r.status_code))
    print("reason code",str(r.reason))
'''

'\nsession = requests.Session()\nbase_url = "https://eodhistoricaldata.com/api/fundamentals/GSPC.INDX"\napi_token = "605766cf49ac93.89487471"\nparams = {\'api_token\': api_token}\nr = session.get(base_url, params=params)\nif r.status_code == requests.codes.ok:\n    #df = pd.read_csv(StringIO(r.text), skipfooter=0, parse_dates=[0], index_col=0, engine=\'python\')\n    df = pd.read_csv(StringIO(r.text), skipfooter=0,index_col=0)\n\n    \nelse:\n    print("status code",str(r.status_code))\n    print("reason code",str(r.reason))\n'

In [22]:
# with open("sample.json", "w") as outfile:
#    outfile.write(json_object)
'''
json_object = json.dumps(r.text, indent = 4)
with open("gspc.json", "w") as outfile:
    outfile.write(json_object)
    '''

'\njson_object = json.dumps(r.text, indent = 4)\nwith open("gspc.json", "w") as outfile:\n    outfile.write(json_object)\n    '